In [23]:
# Import des libraries

from bigml.api import BigML
import pandas
from pandas import read_csv
import matplotlib

In [2]:
# Modification du fichier Train de Kaggle

df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv', index_col=0)
df = df.fillna(0)
df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
df.loc[df.age > 80, 'isOld'] = '1' 
df.loc[df.age <= 80, 'isOld'] = '0' 
df.to_csv('train_full.csv')

In [3]:
# Modification du fichier Test de Kaggle

df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-test.csv', index_col=0)
df = df.fillna(0)
df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
df.loc[df.age > 80, 'isOld'] = '1' 
df.loc[df.age <= 80, 'isOld'] = '0' 
df.to_csv('test_full.csv')

In [4]:
# Connexion au projet via l'API de BIG ML

api = BigML('MAXIMEFAVOT', '69757fb108f0bd7a8678e0b66e3de66ae6478c81', project='project/5d9e081659f5c320e1000597')

In [5]:
# Creation des sources sur BigML

source_train = api.create_source('train_full.csv')
trainfull_dataset = api.create_dataset(source_train)

source_test = api.create_source('test_full.csv')
testfull_dataset = api.create_dataset(source_test)

In [9]:
# Création du split

train_dataset = api.create_dataset(
    trainfull_dataset, {"name": "Training_split", "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dataset(
    trainfull_dataset, {"name": "Test_split", "sample_rate": 0.8, "seed": "my seed", "out_of_bag": True})

You have reached the max number of parallel tasks
You'll need to buy some more credits to perform the chosen action


2019-11-27 15:55:45,990: You have reached the max number of parallel tasks
You'll need to buy some more credits to perform the chosen action


In [10]:
# Création du modele Ensemble

ensemble = api.create_ensemble(train_dataset, {"name": "ensemble", "objective_field": "SeriousDlqin2yrs"})

In [15]:
# Batch Prediction et Download

batch_prediction = api.create_batch_prediction(ensemble, testfull_dataset,{"all_fields": True, "probabilities": True})

You have reached the max number of parallel tasks
You'll need to buy some more credits to perform the chosen action


2019-11-27 15:57:19,423: You have reached the max number of parallel tasks
You'll need to buy some more credits to perform the chosen action


In [16]:
# Récuperation des evaluations sur l'API

evaluation = api.create_evaluation(ensemble, test_dataset)
api.ok(evaluation)
api.pprint(evaluation['object']['result']['model']['accuracy'])
api.pprint(evaluation['object']['result']['model']['average_area_under_roc_curve'])
api.pprint(evaluation['object']['result']['model']['confusion_matrix'])

ValueError: Failed to parse a resource string or structure.

In [17]:
api.download_batch_prediction(batch_prediction, filename='Prediction_kaggle.csv')

ValueError: Failed to parse a resource string or structure.

In [18]:
# Mise en forme pour soumission Kaggle

df = read_csv('Prediction_kaggle.csv', index_col=0)
df2 = df[['1 probability']]
df2.index.names = ['Id'] 
df2.rename(columns={"1 probability": "Probability"}, inplace= True)
df2.to_csv('Kaggle-submission.csv')

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [24]:
# Soumission Kaggle

import kaggle
submission_file = "Kaggle-submission.csv"
kaggle.api.competition_submit(submission_file, "BigML ensemble", "GiveMesomeCredit")

100%|██████████| 1.66M/1.66M [00:03<00:00, 472kB/s] 


Successfully submitted to Give Me Some Credit